In [1]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd

torch.backends.cudnn.deterministic = True #???

import numpy as np
import matplotlib as mpl

# Sentiment Classifier

### General settings

In [2]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device("cpu") #"cuda:1" if avaialble 

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_CLASSES = 2

### Old dataset code

In [5]:
# import urllib.request
# url = "https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz"
# filename = "movie_data.csv.gz"
# urllib.request.urlretrieve(url,filename)

# # #!python -m wget https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz wget doesnt work (natively works on mac) need to instal module which conda doesnt do right now


# #https://ai.standord.edu/~amaas/data/sentiment/ another source of hte dataset


In [4]:
# import gzip
# import shutil

# with gzip.open("movie_data.csv.gz","rb") as f_in: #"rb" = read bianry mode
#         with open("movie_data.csv", "wb") as f_out: #write binary
#                 shutil.copyfileobj(f_in,f_out)

In [3]:
# df = pd.read_csv("movie_data.csv")
# df.head()

In [2]:
# df.columns = ["TEXT_COLUMN_NAME","LABEL_COLUMN_NAME"]
# df.to_csv("movie_data.csv", index = None)

# df = pd.read_csv("movie_data.csv")
# df.head()

# Prepare Dataset with Torchtext

# new code

In [32]:
from torchtext.datasets import IMDB

# Creare shardingFilterIterDataPipe objects - dericed from DataPipe class
# to print out raw data can use for look
train_datapipe = IMDB(split = "train")
next(iter(train_datapipe))
#[i for i in iter(train_datapipe)]

#list(train_datapipe)

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [16]:
#Tokenizer
from torchtext.data.utils import get_tokenizer
tokenizer = get_tokenizer("basic_english")

#example
tokenizer("Hello my name is Bruno!")


['hello', 'my', 'name', 'is', 'bruno', '!']

# build a vocab class

In [17]:
from torchtext.vocab import build_vocab_from_iterator

#build vocab on train data

#chops up a data iterator into its tokens
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

def get_vocab(train_datapipe):
    vocab = build_vocab_from_iterator(yield_tokens(train_datapipe),
        specials = ["<UNK>","<PAD>"], #?unknown and pad
        max_tokens = 20000) #set max vocab to prevent overfitting
    vocab.set_default_index(vocab["<UNK>"])
    return vocab

vocab = get_vocab(train_datapipe)

In [18]:
vocab.get_itos()

['<UNK>',
 '<PAD>',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 "'",
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 's',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but',
 'film',
 ')',
 '(',
 'you',
 't',
 'on',
 'not',
 'he',
 'are',
 'his',
 'have',
 'be',
 'one',
 '!',
 'all',
 'at',
 'they',
 'by',
 'an',
 'who',
 'from',
 'so',
 'like',
 'there',
 'her',
 'or',
 'just',
 'about',
 'if',
 'has',
 'out',
 'what',
 '?',
 'some',
 'good',
 'can',
 'more',
 'when',
 'she',
 'very',
 'even',
 'my',
 'no',
 'up',
 'would',
 'time',
 'which',
 'only',
 'story',
 'really',
 'their',
 'see',
 'had',
 'we',
 'were',
 'me',
 'than',
 'well',
 'much',
 '-',
 'get',
 'been',
 'people',
 'will',
 'also',
 'other',
 'bad',
 'into',
 'do',
 'because',
 'great',
 'first',
 'how',
 'him',
 'don',
 'most',
 'its',
 'made',
 'then',
 'them',
 'way',
 'could',
 'make',
 'too',
 'any',
 'after',
 'movies',
 'think',
 'characters',
 'character',
 'watch',
 'films',
 'two',
 'many',
 'seen',
 'being'

# create the DataLoader

In [33]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []

    for (_label, _text_) in batch:
        label_list.append(label_transform(_label))
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
    
    return torch.tensor(label_list), pad_sequence(text_list, padding_value = 3.0)

train_iter = IMDB(split = "train")
train_dataloader = DataLoader(list(train_iter),
                   batch_size = 8,
                   shuffle = True,
                   collate_fn = collate_batch)

# Step 1: **Build Vocabulary**
